In [23]:
all_in_one_dataset_json = "../data/W99_R01_all_in_one/all_in_one_dataset.json"
registry_names_dataset_json = '../../../datasets/002_registry_names_dataset/registry_names_dataset.json'

import json
import pandas as pd

all_in_one_dataset = json.load(open(all_in_one_dataset_json))
all_in_one_dataset_dt = pd.read_json(all_in_one_dataset_json).set_index('object_id')
registry_names_dataset = pd.read_json(registry_names_dataset_json).set_index('object_id')


registry_names_dataset.loc[:, "pred_registry_name"] = all_in_one_dataset_dt.b_registry_name

In [34]:
all_in_one_dataset_dt.loc["8db50f47-8302-573d-ab69-0c604008160d"]

pmid                                                         8156512.0
title                Mortality in young first-degree relatives of p...
abstract             Mortality and cancer deaths among young family...
a_registry_name                                     Polyposis Registry
b_registry_name                            Authors' Polyposis Registry
simple_matching                                                  False
llm_response                                                 different
llm_justification    The addition of 'Authors'' in the second regis...
Name: 8db50f47-8302-573d-ab69-0c604008160d, dtype: object

In [39]:
registry_names_dataset.loc[:, "is_correct"] = registry_names_dataset.registry_name == registry_names_dataset.pred_registry_name
is_correct_by_llm = all_in_one_dataset_dt.llm_response == 'same'

registry_names_dataset.loc[is_correct_by_llm, "is_correct"] = True

cln_registry_name = registry_names_dataset.registry_name.str.lower().str.replace(r'[^a-z]', '').str.strip()
cln_pred_registry_name = registry_names_dataset.pred_registry_name.str.lower().str.replace(r'[^a-z]', '').str.strip()
is_correct_partially = []
for i, (r, p) in enumerate(zip(cln_registry_name, cln_pred_registry_name)):
    if r in p or p in r:
        is_correct_partially.append(True)
    else:
        is_correct_partially.append(False)

registry_names_dataset.loc[is_correct_partially, "is_correct"] = True

is_correct
True     194
False    106
Name: count, dtype: int64

In [ ]:
precision = registry_names_dataset.is_correct.value_counts(normalize=True, dropna=False).loc[True]*100

none_names = ["Not specified", "NONE"]
none_mask = registry_names_dataset.registry_name.isin(none_names)
precision_not_none = \
    registry_names_dataset.loc[~none_mask]\
            .is_correct.value_counts(normalize=True, dropna=False).loc[True]*100

print(f"Precision: {precision:.2f}%")
print(f"Number of NONE names: {none_mask.sum()} | {none_mask.sum()/len(none_mask)*100:.2f}%")
print("Removing NONE names:")
print(f"Precision: {precision_not_none:.2f}%")
false_none_preds = registry_names_dataset.pred_registry_name.isin(none_names) & ~none_mask
print(f"Number of false NONE predictions: {false_none_preds.sum()} | {false_none_preds.sum()/len(false_none_preds)*100:.2f}%")


Precision: 64.67%
Number of NONE names: 121 | 40.33%
Removing NONE names:
Precision: 74.86%
Number of false NONE predictions: 39 | 13.00%


In [46]:
registry_names_dataset.loc[none_mask].is_correct.value_counts(normalize=True)

is_correct
False    0.504132
True     0.495868
Name: proportion, dtype: float64

In [40]:
pd.set_option('display.max_rows', 500)

registry_names_dataset[~registry_names_dataset.is_correct][[
    'registry_name', 'pred_registry_name', 'is_correct'
]]

,registry_name,pred_registry_name,is_correct
object_id,,,
003188f2-47f3-5bf5-bb9d-267c4569597a,Not specified,NONE,False
03815b4b-61f1-5273-877e-5edd222b1cf5,Not specified,NONE,False
080245f8-1f9e-5376-ba99-cb452ce04c8f,Not specified,NONE,False
096e6ba6-8400-566e-83d4-668161feb1a4,Medical Birth Registry of Norway,NONE,False
1136f5b9-206e-52d9-890b-93b2109fe9f9,Swedish Perioperative Registry,NONE,False
11dde02a-1d03-5f9b-8833-77a928d952b1,Florida Agency for Health Care Administration ...,NONE,False
123355ed-9c08-5d98-bcc3-b25db6f5d181,National Pupil Database,NONE,False
155921c3-5653-518c-94a1-c8f345039864,Not specified,NONE,False
15d99e64-c396-5ebe-a38f-16533309b26b,Japan Collaborative Cohort Study (JACC Study),NONE,False


In [29]:
registry_names_dataset.is_correct.value_counts()

is_correct
True     193
False    107
Name: count, dtype: int64

In [ ]:
registry_names_dataset.registry_name.value_counts()

In [ ]:
registry_names_dataset[false_none_preds][[
    "registry_name",
    "pred_registry_name"
]]

In [ ]:
registry_names_dataset.query("registry_name != 'NONE'").is_correct.value_counts(normalize=True, dropna=False)

In [ ]:
registry_names_dataset.query("annotated_by == 'ghinwa'").is_correct.value_counts(normalize=True, dropna=False)

In [ ]:
registry_names_dataset\
    .query("is_correct == False")\
    [['registry_name', 'pred_registry_name']].sort_values("registry_name")